# Free Trial Performance Analysis in PostgreSQL
This notebook uses SQL to examine marketing data for a free trial of a subscription product. It analyzes the effectiveness of the free trial in converting new customers into paid subscribers by examining the behavioral trends of trial holders.

### The Data:
The dataset is comprised of synthetic data that represents a 1-month free trial product given to potential customers. The Free Trial records instances of customers beginning a free trial. Following the completion of the trial, customers who successfully convert to a paid subscription are marked by a purchase record in the Purchases table.

The relevant data is contained in two tables:

### Free Trials:
A list of instances of free trial starts. 

- Trial ID: An ID unique to the Free Trial.
- Free Trial Start Date: The date when the customer began their free trial. 
- Region: The world region where the customer is located. 

### Purchases:
 A list of purches recipts from converted trial holders
 
 - Trial ID: The ID of the free trial (This ID is unique as each trial may have a maximum of 1 purchase associated with it).
 - Purchase Date: The date when the customer made their purchase following the free trial. 
 - Purchase Value: The USD value of the customer's purchase. 
 
 ### Central Buisness Question:
The central business question of this analysis is to identify how much a free trial is worth on average by examining the population of trial users versus the population of paid subscribers.

# Initial Exploratory Analysis

In this section I query the "Free Trials" and "Purchases" tables to identify the aggregated number of free trials and purchases by month.  

In [11]:
-- Group free trials by start date (and order by the same.
SELECT
    DATE_TRUNC('month', free_trial_start_date) AS month,
    COUNT(*) AS num_free_trials
FROM
    trials
GROUP BY
    DATE_TRUNC('month', free_trial_start_date)
ORDER BY
    month;

,month,num_free_trials
0,2022-01-01 00:00:00+00:00,3871
1,2022-02-01 00:00:00+00:00,4054
2,2022-03-01 00:00:00+00:00,5190
3,2022-04-01 00:00:00+00:00,5393
4,2022-05-01 00:00:00+00:00,5688
5,2022-06-01 00:00:00+00:00,6043


In [12]:
-- Group purchases by the month of purchase_date (and order by the same).
SELECT
	DATE_TRUNC('month', purchase_date) AS month,
	COUNT(*) AS num_purchases,
	SUM(purchase_value) AS usd_value
FROM
	purchases
GROUP BY
	DATE_TRUNC('month', purchase_date)
ORDER BY
	month;

,month,num_purchases,usd_value
0,2022-02-01 00:00:00+00:00,1269,253800
1,2022-03-01 00:00:00+00:00,1277,255400
2,2022-04-01 00:00:00+00:00,1730,346000
3,2022-05-01 00:00:00+00:00,3150,315000
4,2022-06-01 00:00:00+00:00,1262,286950
5,2022-07-01 00:00:00+00:00,1322,300500


# Data Aggregation - Velocity Metrics by Month

Here, I extract key metrics from the "Free Trials" and "Purchases" tables, including Free Trial Starts, Purchases, and Gross Merchandise Value, all aggregated by month. I then summarize this information using Common Table Expressions. Finally, I use a LEFT JOIN to merge the monthly purchase data with the free trials data. This approach presents the data in a comprehensive combined results table, allowing for clear and succinct comparison between the different metrics.

In [13]:
-- CTE to aggregate free trial starts per month from the trials table.
WITH free_trials_per_month AS (
    SELECT
        DATE_TRUNC('month', free_trial_start_date) AS month,
        COUNT(*) AS num_free_trials
    FROM
        trials
    GROUP BY
        DATE_TRUNC('month', free_trial_start_date)
),
-- CTE to aggregate number of purchases from the purchases table.
purchases_per_month AS (
    SELECT
        DATE_TRUNC('month', purchase_date) AS month,
        COUNT(*) AS num_purchases,
        SUM(purchase_value) AS usd_value
    FROM
        purchases
    GROUP BY
        DATE_TRUNC('month', purchase_date)
)

-- LEFT JOIN between free_trials_per_month and purchases_per_month. Here 'pt' and 'ft' are aliases for 'free_trials_per_month' and 'purchases_per_month'.

SELECT
    ft.month,
    ft.num_free_trials,
    pt.num_purchases,
    pt.usd_value
FROM
    free_trials_per_month ft
LEFT JOIN
    purchases_per_month pt ON ft.month = pt.month;

,month,num_free_trials,num_purchases,usd_value
0,2022-02-01 00:00:00+00:00,4054,1269.0,253800.0
1,2022-04-01 00:00:00+00:00,5393,1730.0,346000.0
2,2022-03-01 00:00:00+00:00,5190,1277.0,255400.0
3,2022-05-01 00:00:00+00:00,5688,3150.0,315000.0
4,2022-06-01 00:00:00+00:00,6043,1262.0,286950.0
5,2022-01-01 00:00:00+00:00,3871,NaN,NaN


# Data Aggregation - Cohort Metrics by Month

Cohort metrics refine the information provided by velocity analysis by grouping users based on their initial interaction with the free trial product and then tracking their subsequent behavior (e.g., making a purchase) over time. In this section I examine the dataset using cohort metrics, using the specific example of tracking purchases and free trials over time. This method acknowledges that the relationship between a free trial and a purchase it not immediate but develops over a period of time, allowing for a more accurate measurement of conversion rates and user engagement. 

In [14]:
-- -- This query links trials to purchases, aggregates data by month to analyze trial conversions, and calculates the average purchase value per trial, presenting a monthly trend analysis.

-- Retrieve information about free trials and subsequent purchases then use LEFT JOIN on
-- trial_id. 
SELECT
    t.trial_id,
    t.free_trial_start_date,
    t.region,
    p.purchase_date,
    p.purchase_value
FROM
    trials t
LEFT JOIN
    purchases p ON t.trial_id = p.trial_id;

	

,trial_id,free_trial_start_date,region,purchase_date,purchase_value
0,A31021,2022-01-01 00:00:00+00:00,North America,NaT,NaN
1,A31022,2022-01-01 00:00:00+00:00,Oceania,NaT,NaN
2,A31023,2022-01-01 00:00:00+00:00,Europe,NaT,NaN
3,A31024,2022-01-01 00:00:00+00:00,Europe,NaT,NaN
4,A31025,2022-01-01 00:00:00+00:00,North America,NaT,NaN
...,...,...,...,...,...
30234,A63569,2022-06-30 00:00:00+00:00,North America,2022-07-30 00:00:00+00:00,250.0
30235,A63570,2022-06-30 00:00:00+00:00,North America,NaT,NaN
30236,A63571,2022-06-30 00:00:00+00:00,Asia,NaT,NaN
30237,A63572,2022-06-30 00:00:00+00:00,Europe,NaT,NaN


In [15]:
-- Aggregate all data by the month of the Free Trial start, and calculate same metrics as before. 
WITH free_trials_and_purchases AS (
	SELECT
		t.trial_id,
		t.free_trial_start_date, 
		t.region, 
		p.purchase_date, 
		p.purchase_value
	FROM
		trials t
	LEFT JOIN
		purchases p ON t.trial_id = p.trial_id
)

SELECT
	DATE_TRUNC('month', f.free_trial_start_date) AS month, 
	COUNT(*) AS num_free_trials, 
	COUNT(f.purchase_date) AS num_purchases, -- Counting purchase dates to determine the number of purchases. 
	SUM(f.purchase_value) AS usd_value
FROM
	free_trials_and_purchases f
GROUP BY
	1
ORDER BY
	1;


,month,num_free_trials,num_purchases,usd_value
0,2022-01-01 00:00:00+00:00,3871,1269,253800
1,2022-02-01 00:00:00+00:00,4054,1277,255400
2,2022-03-01 00:00:00+00:00,5190,1730,346000
3,2022-04-01 00:00:00+00:00,5393,3150,315000
4,2022-05-01 00:00:00+00:00,5688,1262,286950
5,2022-06-01 00:00:00+00:00,6043,1322,300500


# Free Trial Value Calculation

Using the cohort table I have already created, I will now calculate an average value per free trial for each month. 

In [16]:
-- Creates a dataset linking trials with any resulting purchases by trial ID, including trial start dates and regions. 
WITH free_trials_and_purchases AS (
	SELECT
		t.trial_id,
		t.free_trial_start_date, 
		t.region, 
		p.purchase_date, 
		p.purchase_value
	FROM
		trials t
	LEFT JOIN
		purchases p ON t.trial_id = p.trial_id
),

-- Aggregates free trials and purchases by month, counting trials, purchases, and summing purchase values. 
summary_by_month AS (
	SELECT
		DATE_TRUNC('month', f.free_trial_start_date) AS month,
		COUNT(*) AS num_free_trials, 
		COUNT(f.purchase_date) AS num_purchases, 
		SUM(f.purchase_value) AS usd_value
	FROM
		free_trials_and_purchases f
	GROUP BY
		1
)

-- Retrieves monthly trial and purchase metrics, including a calculated value per free trial, ordered by month. 
SELECT
	s.month, 
	s.num_free_trials, 
	s.num_purchases, 
	s.usd_value, 
	(s.usd_value::FLOAT) / NULLIF(s.num_free_trials, 0)::Float AS cohort_value_per_free_trial
FROM
	summary_by_month s
ORDER BY
	1; 

,month,num_free_trials,num_purchases,usd_value,cohort_value_per_free_trial
0,2022-01-01 00:00:00+00:00,3871,1269,253800,65.564454
1,2022-02-01 00:00:00+00:00,4054,1277,255400,62.999507
2,2022-03-01 00:00:00+00:00,5190,1730,346000,66.666667
3,2022-04-01 00:00:00+00:00,5393,3150,315000,58.409049
4,2022-05-01 00:00:00+00:00,5688,1262,286950,50.448312
5,2022-06-01 00:00:00+00:00,6043,1322,300500,49.726957


# Dimensional Breakdown

Here I breakdown the average value per Free Trial by Region to see how the values differ. 

In [17]:
-- This CTE merges 'trials' with 'purchases' using a LEFT JOIN on trial_id, capturing trial details and any associated purchase data, ensuring a complete view of trial-to-purchase conversion.
WITH free_trials_and_purchases AS (
	SELECT
		t.trial_id,
		t.free_trial_start_date, 
		t.region, 
		p.purchase_date, 
		p.purchase_value
	FROM
		trials t
	LEFT JOIN
		purchases p ON t.trial_id = p.trial_id
),

-- Aggregates trial and purchase data by month and region, calculating total free trials, purchases, and purchase value, enabling detailed analysis of conversion trends. 
summary_by_month AS (
	SELECT
		DATE_TRUNC('month', f.free_trial_start_date) AS month, 
		f.region,
		COUNT(*) AS num_free_trials, 
		COUNT(f.purchase_date) AS num_purchases, 
		SUM(f.purchase_value) AS usd_value
	FROM
		free_trials_and_purchases f
	GROUP BY
		1, 2
)

-- Selects monthly regional data on free trials, purchases, total value, and calculates the average value per free trial, ordered by month and region. 
SELECT
	s.month, 
	s.region, 
	s.num_free_trials, 
	s.num_purchases, 
	s.usd_value,
	(s.usd_value::FLOAT) / NULLIF(s.num_free_trials, 0)::FLOAT AS cohort_value_per_free_trial
FROM
	summary_by_month s
ORDER BY
	1, 2;
	



,month,region,num_free_trials,num_purchases,usd_value,cohort_value_per_free_trial
0,2022-01-01 00:00:00+00:00,Africa,653,85,17000,26.033691
1,2022-01-01 00:00:00+00:00,Asia,732,244,48800,66.666667
2,2022-01-01 00:00:00+00:00,Europe,989,368,73600,74.418605
3,2022-01-01 00:00:00+00:00,North America,1016,412,82400,81.102362
4,2022-01-01 00:00:00+00:00,Oceania,350,141,28200,80.571429
5,2022-01-01 00:00:00+00:00,South America,131,19,3800,29.007634
6,2022-02-01 00:00:00+00:00,Africa,743,81,16200,21.803499
7,2022-02-01 00:00:00+00:00,Asia,800,248,49600,62.000000
8,2022-02-01 00:00:00+00:00,Europe,1030,381,76200,73.980583
9,2022-02-01 00:00:00+00:00,North America,995,393,78600,78.994975
